# Hands-on Introduction to `regex`

In this lesson, we'll learn about a useful tool in the NLP toolkit: regex.

Let's consider two motivating examples:

#### 1. The phone number problem

Suppose we are given some data that includes phone numbers:

123-456-7890

123 456 7890

101 Howard

Some of the phone numbers have different formats (hyphens, no hyphens).  Also, there are some errors in the data-- 101 Howard isn't a phon number!  How can we find all the phone numbers?

#### 2. Creating our own tokens

In the previous lessons, we used sklearn or fastai to tokenize our text.  What if we want to do it ourselves?

## 1. The phone number problem

Suppose we are given some data that includes phone numbers:

123-456-7890

123 456 7890

(123)456-7890

101 Howard

Some of the phone numbers have different formats (hyphens, no hyphens, parentheses).  Also, there are some errors in the data-- 101 Howard isn't a phone number!  How can we find all the phone numbers?

We will attempt this without regex, but will see that this quickly leads to lot of if/else branching statements and isn't a veyr promising approach:

### Without `regex`

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *
from fastai.utils.mem import GPUMemTrace #call with mtrace

In [5]:
import string

#### Here are some example strings. We would like to write a function to decide whether or not they are valid phone numbers.

In [23]:
phone1 = "123-456-7890"

phone2 = "123 456 7890"

not_phone1 = "101 Howard"

In [7]:
string.digits

'0123456789'

#### Check that the string only contains digits, or the characters -, (, )

In [27]:
def check_phone(inp):
    valid_chars = string.digits + ' -()'
    for char in inp:
        if char not in valid_chars:
            return False
    return True

In [28]:
assert(check_phone(phone1))
assert(check_phone(phone2))
assert(not check_phone(not_phone1))

#### So far so good. But this test string 'breaks' our code:  

In [29]:
not_phone2 = "1234"

In [30]:
assert(not check_phone(not_phone2))
# check_phone(not_phone2)

AssertionError: 

#### Ok, let's improve our function to handle this edge case: we'll make sure the string has exactly 10 digits.

In [360]:
def check_phone(inp):
    nums = string.digits
    valid_chars = nums + ' -()'
    num_counter = 0
    for char in inp:
        if char not in valid_chars:
            return False
        if char in nums:
            num_counter += 1
    if num_counter==10:
        return True
    else:
        return False

In [361]:
assert(check_phone(phone1))
assert(check_phone(phone2))
assert(not check_phone(not_phone1))
assert(not check_phone(not_phone2))

#### But what about these examples?

In [363]:
# This is correctly rejected
not_phone3 = '34!NA5098gn#213ee2'
assert(not check_phone(not_phone3))

In [364]:
# This is not rejected
not_phone4 = "34 50 98 21 32"
assert(not check_phone(not_phone4))

AssertionError: 

In [366]:
# This is not rejected
not_phone5 = "(34)(50)()()982132"
assert(not check_phone(not_phone4))

AssertionError: 

This is getting increasingly unwieldy.  We need a different approach.

## 2. Introducing `regex`

Useful regex resources:

- https://regexr.com/
- http://callumacrae.github.io/regex-tuesday/
- https://regexone.com/

**Best practice: Be as specific as possible.**

Parts of the following section were adapted from Brian Spiering, who taught the MSDS [NLP elective last summer](https://github.com/brianspiering/nlp-course).

### What is regex?

Regular expressions is a pattern matching language. 

Instead of writing `0 1 2 3 4 5 6 7 8 9`, you can write `[0-9]` or `\d`

It is Domain Specific Language (DSL). Powerful (but limited language). 

**What other DSLs do you already know?**
- SQL  
- Markdown
- TensorFlow

### Matching Phone Numbers (The "Hello, world!" of Regex)

`[0-9][0-9][0-9]-[0-9][0-9][0-9]-[0-9][0-9][0-9][0-9]` matches US telephone number.

Refactored: `\d\d\d-\d\d\d-\d\d\d\d`

A **metacharacter** is one or more special characters that have a unique meaning and are NOT used as literals in the search expression. For example "\d" means any digit.

**Metacharacters are the special sauce of regex.**

### Quantifiers
-----

Allow you to specify how many times the preceding expression should match. 

`{}` is an exact qualifer

Refactored: `\d{3}-\d{3}-\d{4}`

### Unexact quantifiers

1. `?` question mark - zero or one 
2. `*` star - zero or more
3. `+` plus sign - one or more | 

### Regex can look really weird, since it's so concise

The best (only?) way to learn it is through practice.  Otherwise, you feel like you're just reading lists of rules.

Let's take 15 minutes to begin working through the lessons on [regexone](https://regexone.com/).

**Reminder: Be as specific as possible!**

### Pros & Cons of Regex

**What are the advantages of regex?**

1. Concise and powerful pattern matching DSL
2. Supported by many computer languages, including SQL

**What are the disadvantages of regex?**

1. Brittle 
2. Hard to write, can get complex to be correct
3. Hard to read

### Dictionary of regex metacharacters

note: in the following, "character" can refer to a single character or a group of characters

\ make following meta-character into a literal <br>
\d match one instance of digit<br>
\D match one instance of non-digit<br>
\w match one instance of alphanumeric<br>
\W match one instance of non-alphanumeric (symbol)<br>
\s match one instance of whitespace<br>
\S match one instance of non-whitespace<br>

. match one instance of any character (wildcard match)<br>
? match 0 or 1 instance of preceding character (optional match)<br>


\+ match 1 or more instance of preceding character (mandatory match)<br>
\* match 0 or more instance of preceding character (optional match)<br>

c{3} match a string which contains 3 instances of c, the preceding character<br>
c{m,n} Match a string in which the preceding character c occurs at least m times, and at most n times.

[] encloses an expression that selects exactly one instance of a character class:
[a-z] match exactly one instance of any of the characters a through z<br>
^ is a regex character with a split personality (there might be others)
^ when occurring inside brackets [], ^ means "not", i.e. <br>
[^a-z] match exactly one instance of a character that is not a through z<br>

otherwise, ^ refers to the character at the beginning of a string: <br>
^A match A if it is the first character in a string<br>

x\\$ refers to the character x at the end of a string: <br>
\\.\\$ match a period if it is the last character in a string<br>
z\\$ match a z if it is the last character in a string<br>
\w\\$ match an alphanumeric if it is the last character in a string <br>

(dog) match a string group containing the characters d, o, g in that order <br>

<img src="regex.png" alt="regex metacharacters" style="width: 30%"/>

ref: - https://regexone.com/

## 3. Tokenizers revisited: using regex to build our own tokenizer

In the previous lessons, we used a tokenizer.  Now, let's learn how we could do this ourselves, and get a better understanding of tokenization.

What if we needed to create our own tokens? <br>First we need to access `regex`, which lives in the  `re` package  

In [63]:
import re

### The tokenizer needs to find sequences of characters that are words or numbers, and separate out the punctuation marks, and endings

#### Note about the code cell below: r"*string*" is a `python` (not `regex`) command that tells the string interpreter that the *string* enclosed between the quotes is a *raw text expression*, meaning that python's string metacharacters $^{\dagger}$ are interpreted  as `literals` rather than being parsed with their `meta` superpowers. <br>
$^{\dagger}$ such as escape sequences like \\' and \\" that also encode the \' and \" characters and can be used inside a string.


In [338]:
# capture punctuation, including python escape sequences
re_punc = re.compile(r"([\"\''().,;:/_?!—\-])") 

# capture the n't ending
re_apos = re.compile(r"n ' t ")

#capture the 's ending
re_bpos = re.compile(r" ' s ")

# Note that there are other word endings that we could have added, such as the
# 'd in you'd, I'd, we'd
# 'll in I'll you'll we'll

# capture sequences of multiple blank spaces (we'll replace these with a single blank space)
re_mult_space = re.compile(r" +") 

# Here is a function that takes as input any text string (called sent)
#     the awkward construction is so we can access all the intermediate results
def simple_toks(text):
    # substitions to be made
    text0 = re_punc.sub(" ", text)
    text1 = re_punc.sub(r" \1 ", text)
    text2 = re_apos.sub(r" n't ", text1)
    text3 = re_bpos.sub(r" 's ", text2)
    text4 = re_mult_space.sub(' ', text3)
    text5 = text4.lower()
    text6 = text5.split()
    return text0, text1, text2, text3, text4, text5, text6

### Let's test simple_tok() to see if it properly tokenizes a sentence that includes lots of punctuation:

In [339]:
text = r"I don't know who Kara's 10 new friend(s) are; -- is one of them 'Mr./Mrs. Toad'?"

#### Did we get the desired output?

In [340]:
simple_toks(text)[6]

['i',
 'do',
 "n't",
 'know',
 'who',
 'kara',
 "'s",
 '10',
 'new',
 'friend',
 '(',
 's',
 ')',
 'are',
 ';',
 '-',
 '-',
 'is',
 'one',
 'of',
 'them',
 "'",
 'mr',
 '.',
 '/',
 'mrs',
 '.',
 'toad',
 "'",
 '?']

### As a final bit of pre-processing, `join` the `list` of `tokens` into a single `string`, using a  whitespace `separator`

In [341]:
' '.join(simple_toks(text)[6])

"i do n't know who kara 's 10 new friend ( s ) are ; - - is one of them ' mr . / mrs . toad ' ?"

In [342]:
text0, text1, text2, text3, text4, text5, text6 = simple_toks(text)

In [350]:
# in text substitute a blank character for every punctuation character
text0

'I don t know who Kara s 10 new friend s  are     is one of them  Mr  Mrs  Toad  '

In [351]:
# in text, add spaces around every captured punctuation character
text1

"I don ' t know who Kara ' s 10 new friend ( s )  are ;   -  -  is one of them  ' Mr .  / Mrs .  Toad '  ? "

In [345]:
# in text1, collapse " n ' t" to the "n't" ending surrounded by spaces)
text2

"I do n't know who Kara ' s 10 new friend ( s )  are ;   -  -  is one of them  ' Mr .  / Mrs .  Toad '  ? "

In [346]:
# in text2, collapse " ' s " to the "'s" ending, surrounded by spaces
text3

"I do n't know who Kara 's 10 new friend ( s )  are ;   -  -  is one of them  ' Mr .  / Mrs .  Toad '  ? "

In [352]:
# in text3, collapse all strings of multiple spaces to one space
text4

"I do n't know who Kara 's 10 new friend ( s ) are ; - - is one of them ' Mr . / Mrs . Toad ' ? "

In [348]:
# lower case text4 
text5

"i do n't know who kara 's 10 new friend ( s ) are ; - - is one of them ' mr . / mrs . toad ' ? "

In [353]:
# split text5 on blank space to create a list of tokens, the final result
text6

['i',
 'do',
 "n't",
 'know',
 'who',
 'kara',
 "'s",
 '10',
 'new',
 'friend',
 '(',
 's',
 ')',
 'are',
 ';',
 '-',
 '-',
 'is',
 'one',
 'of',
 'them',
 "'",
 'mr',
 '.',
 '/',
 'mrs',
 '.',
 'toad',
 "'",
 '?']

### Now try the tokenizer on a list of sentences.

In [42]:
sentences = ['All this happened, more or less.',
             'The war parts, anyway, are pretty much true.',
             "One guy I knew really was shot for taking a teapot that wasn't his.",
             'Another guy I knew really did threaten to have his personal enemies killed by hired gunmen after the war.',
             'And so on.',
             "I've changed all their names."]

###  the `map` function applies the simple_toks function to the input list of sentences, producing an object of the `map` class, which can be acted upon by list() to produce a `list` object

In [358]:
print(type(map(simple_toks, sentences)))

<class 'map'>


In [359]:
print(type(list(map(simple_toks, sentences))))

<class 'list'>


In [43]:
tokens = list(map(simple_toks, sentences))

In [44]:
tokens

[['all', 'this', 'happened', ',', 'more', 'or', 'less', '.'],
 ['the',
  'war',
  'parts',
  ',',
  'anyway',
  ',',
  'are',
  'pretty',
  'much',
  'true',
  '.'],
 ['one',
  'guy',
  'i',
  'knew',
  'really',
  'was',
  'shot',
  'for',
  'taking',
  'a',
  'teapot',
  'that',
  'was',
  "n't",
  'his',
  '.'],
 ['another',
  'guy',
  'i',
  'knew',
  'really',
  'did',
  'threaten',
  'to',
  'have',
  'his',
  'personal',
  'enemies',
  'killed',
  'by',
  'hired',
  'gunmen',
  'after',
  'the',
  'war',
  '.'],
 ['and', 'so', 'on', '.'],
 ['i', "'", 've', 'changed', 'all', 'their', 'names', '.']]

Once we have our tokens, the next step is to convert them to integer ids.  We will also need to get our vocabulary list, and have a way to convert between the words and their associated ids.

In [45]:
import collections

In [46]:
PAD = 0; SOS = 1

def toks2ids(sentences):
    voc_cnt = collections.Counter(t for sent in sentences for t in sent)
    vocab = sorted(voc_cnt, key=voc_cnt.get, reverse=True)
    vocab.insert(PAD, "<PAD>")
    vocab.insert(SOS, "<SOS>")
    w2id = {w:i for i,w in enumerate(vocab)}
    ids = [[w2id[t] for t in sent] for sent in sentences]
    return ids, vocab, w2id, voc_cnt

In [47]:
ids, vocab, w2id, voc_cnt = toks2ids(tokens)

In [48]:
ids

[[5, 13, 14, 3, 15, 16, 17, 2],
 [6, 7, 18, 3, 19, 3, 20, 21, 22, 23, 2],
 [24, 8, 4, 9, 10, 11, 25, 26, 27, 28, 29, 30, 11, 31, 12, 2],
 [32, 8, 4, 9, 10, 33, 34, 35, 36, 12, 37, 38, 39, 40, 41, 42, 43, 6, 7, 2],
 [44, 45, 46, 2],
 [4, 47, 48, 49, 5, 50, 51, 2]]

In [28]:
vocab

['<PAD>',
 '<SOS>',
 '.',
 ',',
 'i',
 'all',
 'the',
 'war',
 'guy',
 'knew',
 'really',
 'was',
 'his',
 'this',
 'happened',
 'more',
 'or',
 'less',
 'parts',
 'anyway',
 'are',
 'pretty',
 'much',
 'true',
 'one',
 'shot',
 'for',
 'taking',
 'a',
 'teapot',
 'that',
 "n't",
 'another',
 'did',
 'threaten',
 'to',
 'have',
 'personal',
 'enemies',
 'killed',
 'by',
 'hired',
 'gunmen',
 'after',
 'and',
 'so',
 'on',
 "'",
 've',
 'changed',
 'their',
 'names']

Q: what could be another name of the `vocab` variable above?

In [29]:
w2id

{'<PAD>': 0,
 '<SOS>': 1,
 '.': 2,
 ',': 3,
 'i': 4,
 'all': 5,
 'the': 6,
 'war': 7,
 'guy': 8,
 'knew': 9,
 'really': 10,
 'was': 11,
 'his': 12,
 'this': 13,
 'happened': 14,
 'more': 15,
 'or': 16,
 'less': 17,
 'parts': 18,
 'anyway': 19,
 'are': 20,
 'pretty': 21,
 'much': 22,
 'true': 23,
 'one': 24,
 'shot': 25,
 'for': 26,
 'taking': 27,
 'a': 28,
 'teapot': 29,
 'that': 30,
 "n't": 31,
 'another': 32,
 'did': 33,
 'threaten': 34,
 'to': 35,
 'have': 36,
 'personal': 37,
 'enemies': 38,
 'killed': 39,
 'by': 40,
 'hired': 41,
 'gunmen': 42,
 'after': 43,
 'and': 44,
 'so': 45,
 'on': 46,
 "'": 47,
 've': 48,
 'changed': 49,
 'their': 50,
 'names': 51}

What are the uses of RegEx?
---



1. Find / Search
2. Find & Replace
3. Cleaning data by removing unwanted characters
4. Processing text

Don't forgot about Python's `str` methods
-----

`str.<tab>`
    
str.find()

In [54]:
str?

In [61]:
text = '12, 14, 22, 17, 31'
dd = text.split(' ')
dd

['12,', '14,', '22,', '17,', '31']

In [56]:
dir(text)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',


Regex vs. String methods
-----

1. String methods are easier to understand.
1. String methods express the intent more clearly. 

-----

1. Regex handle much broader use cases.
1. Regex can be language independent.
1. Regex can be faster at scale.

## What about unicode?

In [62]:
message = "😒🎦 🤢🍕"

re_frown = re.compile(r"😒|🤢")
re_frown.sub(r"😊", message)

'😊🎦 😊🍕'

## Regex Errors:

__False positives__ (Type I): Matching strings that we should __not__ have
matched

__False negatives__ (Type II): __Not__ matching strings that we should have matched

Reducing the error rate for a task often involves two antagonistic efforts:

1. Minimizing false positives
2. Minimizing false negatives

**Important to have tests for both!**

In a perfect world, you would be able to minimize both but in reality you often have to trade one for the other.

Useful Tools:
----
- [Regex cheatsheet](http://www.cheatography.com/davechild/cheat-sheets/regular-expressions/)
- [regexr.com](http://regexr.com/) Realtime regex engine
- [pyregex.com](https://pythex.org/) Realtime Python regex engine

Summary
----

1. We use regex as a metalanguage to find string patterns in blocks of text
1. `r""` are your IRL friends for Python regex
1. We are just doing binary classification so use the same performance metrics
1. You'll make a lot of mistakes in regex 😩. 
    - False Positive: Thinking you are right but you are wrong
    - False Negative: Missing something

<center><img src="images/face_tat.png" width="700"/></center>

<br>
<br>
---

<center><img src="https://imgs.xkcd.com/comics/perl_problems.png" width="700"/></center>

<center><img src="https://imgs.xkcd.com/comics/regex_golf.png" width="700"/></center>

Regex Terms
----


- __target string__:	This term describes the string that we will be searching, that is, the string in which we want to find our match or search pattern.


- __search expression__: The pattern we use to find what we want. Most commonly called the regular expression. 


- __literal__:	A literal is any character we use in a search or matching expression, for example, to find 'ind' in 'windows' the 'ind' is a literal string - each character plays a part in the search, it is literally the string we want to find.

- __metacharacter__: A metacharacter is one or more special characters that have a unique meaning and are NOT used as literals in the search expression. For example "." means any character.

Metacharacters are the special sauce of regex.

- __escape sequence__:	An escape sequence is a way of indicating that we want to use a metacharacters as a literal. 

In a regular expression an escape sequence involves placing the metacharacter \ (backslash) in front of the metacharacter that we want to use as a literal. 

`'\.'` means find literal period character (not match any character)

Regex Workflow
---
1. Create pattern in Plain English
2. Map to regex language
3. Make sure results are correct:
    - All Positives: Captures all examples of pattern
    - No Negatives: Everything captured is from the pattern
4. Don't over-engineer your regex. 
    - Your goal is to Get Stuff Done, not write the best regex in the world
    - Filtering before and after are okay.